In [1]:
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.0/299.0 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt

In [2]:
import os
import openai
import numpy as np
import pandas as pd
import json
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding

In [5]:
openai.api_key = ""
# or
os.environ['OPENAI_API_KEY'] = ""

In [6]:
System_Prompt = """"You are a professional news article summarizer, trained to provide clear, concise, and informative summaries of news articles. Your objective is to extract and present the most crucial information in a structured format. Follow the steps below:

Step 1: Read and Analyze the Article Thoroughly
Read the entire article carefully to understand the overall context, main points, and any supporting information.
Pay attention to the 5Ws (Who, What, When, Where, Why) and the How. Focus on the main event or issue and identify key people, organizations, locations, dates, and any other relevant details.
Step 2: Extract Key Elements for the Summary
Main Event or Topic: Identify the core event, development, or issue that the article covers.
Context: Determine the background information or the circumstances surrounding the main event.
Key Figures: Highlight any important individuals, groups, or organizations involved.
Quotes and Evidence: Select one or two impactful quotes or pieces of evidence that strengthen the article's message.
Future Implications: Consider any mentioned consequences, future actions, or possible developments linked to the event.
Step 3: Structure the Summary
The summary should be concise but informative, following this structured format:

Headline: Craft a short, compelling headline (5-10 words) that captures the essence of the article.
Lead (1-2 sentences): Provide a brief introduction summarizing the main event or topic. Aim to cover the ‘What’ and ‘Who’ aspects here.
Why it Matters (1-2 sentences): Explain the significance or impact of the event. Why should the reader care about this news?
Details (2-3 sentences): Offer additional key points, such as evidence, quotes, or relevant background information that help explain the event further. Ensure this section includes important facts like ‘When’ and ‘Where.’
Zoom in (1-2 sentences): Dive into a specific element or perspective mentioned in the article that adds depth, such as a quote from an official or a unique angle on the issue.
Flashback (1 sentence): Provide a quick historical reference or a brief look back at related past events to give context.
Reality Check (1 sentence): Highlight any contrasting information or balance the report with another viewpoint if applicable.
Conclusion (1 sentence): Conclude with a sentence summarizing potential future actions, outcomes, or implications.
Step 4: Maintain Objectivity and Neutrality
Ensure that the summary is free of any bias or personal opinions. Present the information factually, with clarity and neutrality.
Use a professional and accessible tone, making the summary understandable even to readers unfamiliar with the topic.
Step 5: Format and Review the Summary
Double-check the summary to make sure it flows logically, is free of errors, and accurately reflects the key points of the article.
Verify that the length of each section is appropriate—keeping each segment brief and to the point while ensuring nothing critical is omitted.
Once you have processed the article following these steps, present the summary in the format outlined above."""

In [7]:
user_message = """
Researchers studying sperm whale communication say they've uncovered sophisticated structures similar to those found in human language.

In the inky depths of the midnight zone, an ocean giant bears the scars of the giant squid she stalks. She searches the darkness, her echolocation pulsing through the water column. Then she buzzes – a burst of rapid clicks – just before she goes in for the kill.

But exactly how sperm whales catch squid, like many other areas of their lives, remains a mystery. "They're slow swimmers," says Kirsten Young, a marine scientist at the University of Exeter. Squid, on the other hand, are fast. "How can [sperm whales] catch squid if they can only move at 3 knots [5.5 km/h or 3.5mph]? Are the squid moving really slowly? Or are the whales stunning them with their vocalisations? What happens down there? Nobody really knows," she says.


Sperm whales are not easy to study. They spend much of their lives foraging or hunting at depths beyond the reach of sunlight. They are capable of diving over 3km (10,000ft) and can hold their breath for two hours.

"At 1000m (3300ft) deep, many of the group will be facing the same way, flanking each other – but across an area of several kilometres," says Young. "During this time they're talking, clicking the whole time." After about an hour, she says, the group rises to the surface in synchrony. "They'll then have their rest phase. They might be at the surface for 15 to 20 minutes. Then they'll dive again," she says.


At the end of a day of foraging, says Young, the sperm whales come together at the surface and rub against each other, chatting while they socialise. "As researchers, we don't see a lot of their behaviour because they don't spend that much time at the surface," she says. "There's masses we don't know about them, because we are just seeing a tiny little snapshot of their lives during that 15 minutes at the surface."


It was around 47 million years ago that land-roaming cetaceans began to gravitate back towards the ocean – that's 47 million years of evolution in an environment alien to our own. How can we hope to easily understand creatures that have adapted to live and communicate under such different evolutionary pressures to ourselves?


"It's easier to translate the parts where our world and their world overlap – like eating, nursing or sleeping," says David Gruber, lead and founder of the Cetacean Translation Initiative (Ceti) and professor of biology at the City University of New York. "As mammals, we share these basics with others. But I think it's going to get really interesting when we try to understand the areas of their world where there's no intersection with our own," he says.

Now, from elephants to dogs, modern technology is helping researchers to sift through enormous datasets, and uncover previously unknown diversity and complexity in animal communication. And Ceti's researchers say they, too, have used AI to decode a "sperm whale phonetic alphabet".

In 2005, Shane Gero, biology lead for Ceti, founded The Dominica Sperm Whale Project to study the social and vocal behaviour of around 400 sperm whales that live in the Eastern Caribbean. Almost 20 years – and thousands of hours of observation – later, the researchers have discovered intricacies in whale vocalisations never before observed, revealing structures within sperm whale communication akin to human language.

We're at base camp. This is a new place for humans to be – David Gruber
Sperm whales live in multi-level, matrilineal societies – groups of daughters, mothers and grandmothers – while the males roam the oceans, visiting the groups to breed. They are known for their complex social behaviour and group decision-making, which requires sophisticated communication. For example, they are able to adapt their behaviour as a group when protecting themselves from predators like orcas or humans.


Sperm whales communicate with each other using rhythmic sequences of clicks, called codas. It was previously thought that sperm whales had just 21 coda types. However, after studying almost 9,000 recordings, the Ceti researchers identified 156 distinct codas. They also noticed the basic building blocks of these codas which they describe as a "sperm whale phonetic alphabet" – much like phonemes, the units of sound in human language which combine to form words. (Watch the video below to hear some of the variety in sperm whale vocalisations the AI identified.)

Pratyusha Sharma, a PhD student at MIT and lead author of the study, describes the "fine-grain changes" in vocalisations the AI identified. Each coda consists of between three and 40 rapid-fire clicks. The sperm whales were found to vary the overall speed, or the "tempo", of the codas, as well as to speed up and slow down during the delivery of a coda, in other words, making it "rubato". Sometimes they added an extra click at the end of a coda, akin, says Sharma, to "ornamentation" in music. These subtle variations, she says, suggest sperm whale vocalisations could carry a much richer amount of information than previously thought.


"Some of these features are contextual," says Sharma. "In human language, for example, I can say 'what' or 'whaaaat!?'. It's the same word, but to understand the meaning you have to listen to the whole sound," she says.


The researchers also found the sperm whale "phonemes" could be used in a combinatorial fashion, allowing the whales to construct a vast repertoire of distinct vocalisations. The existence of a combinatorial coding system, write the report authors, is a prerequisite for "duality of patterning" – a linguistic phenomenon thought to be unique to human language – in which meaningless elements combine to form meaningful words.

However, Sharma emphasises, this is not something they have any evidence of as yet. "What we show in sperm whales is that the codas themselves are formed by combining from this basic set of features. Then the codas get sequenced together to form coda sequences." Much like humans combine phonemes to create words, and then words to create sentences.


So, what does all this tell us about sperm whales' intelligence? Or their ability to reason, or store and share information?


"Well, it doesn't tell us anything yet," says Gruber. "Before we can get to those amazing questions, we need to build a fundamental understanding of how [sperm whales communicate] and what's meaningful to them. We see them living very complicated lives, the coordination and sophistication in their behaviours.  We're at base camp. This is a new place for humans to be – just give us a few years. Artificial intelligence is allowing us to see deeper into whale communication than we've ever seen before."


But not everyone is convinced, with experts warning of an anthropocentric focus on language which risks forcing us to view things from one perspective.
"""

In [8]:
struct = [{'role' : 'system', 'content' : System_Prompt}]
struct.append({"role": "user", "content": user_message})
chat = openai.ChatCompletion.create(model="gpt-4o-mini", messages = struct)
response = chat.choices[0].message.content
struct.append({"role": "assistant", "content": response})
print("Assistant:", response)

Assistant: **Headline:** Researchers Decode Sperm Whale Communication Complexity

**Lead:** A new study reveals that sperm whales utilize intricate vocal structures akin to human language, shedding light on their communication methods in the deep ocean.

**Why it Matters:** Understanding sperm whale communication is crucial as it explores the complexities of animal language, potentially reshaping our view of intelligence in non-human species.

**Details:** The Cetacean Translation Initiative (Ceti) discovered 156 distinct coda types, previously thought to be just 21, in sperm whale vocalizations by analyzing nearly 9,000 recordings. The findings, supported by AI technology, suggest these vocalizations contain nuanced information similar to human phonetics.

**Zoom in:** Pratyusha Sharma, a lead author of the study, noted the "fine-grain changes" in vocalizations, highlighting how variations in tempo and additional clicks resemble musical ornamentation, indicating enhanced communicative